# Cuaderno 4: Problema del bin packing

En el problema del *bin packing* (*empacamiento en recipientes*), se tienen dados $n$ objetos, con pesos $w_1, \ldots, w_n$. Se quiere empacar estos objetos empleando la menor cantidad de recipientes de capacidad $b$, de tal forma que la suma de los pesos de los objetos empacados en un recipiente no supere nunca la capacidad de este.

Denotemos por $I$ al conjunto de objetos y por $J$ al conjunto de recipientes. Para garantizar la existencia de al menos una solución factible, supondremos que $w_i \leq b, \forall i \in \{1,\ldots,n\}$, y que $\left|J\right| \geq \left|I\right|$. Utilizaremos dos familias de variables de decisión binarias: 

1. Variables $x_{ij}$, con $i \in I$ y $j \in J$, para indicar si el objeto $i$ es empacado en el recipiente $j$. 
2. Variables $y_j$, con $j \in J$, para indicar si el recipiente $j$ es utilizado en la solución.

El problema puede formularse como el siguiente programa lineal entero:


\begin{align*}
&\min \sum_{j \in J} y_j \\ 
&\mbox{s.r.}\\
& \sum_{j \in J} x_{ij} = 1, \quad \forall i \in I, \\
& \sum_{i \in I} w_i x_{ij} \leq b y_j, \quad \forall j \in J, \\
& x_{ij} \in \{0, 1\}, \quad \forall i \in I, j \in J,\\
& y_{j} \in \{0, 1\}, \quad \forall j \in J.
\end{align*}

La primera familia de restricciones especifica que cada objeto debe ser empacado exactamente en un recipiente.

Las restricciones de la segunda familia se conocen como **restricciones de enforzamiento**. Observar que si $y_j = 0$ (es decir, si el recipiente $j$ no es usado en la solución), entonces el lado derecho se evalúa a cero y todas las variables de la suma en el lado izquierdo deben forzosamente valer cero (es decir, no puede asignarse ningún objeto al recipiente $j$). Por otra parte, si $y_j = 1$, el lado derecho se evalúa a $b$ y en este caso la restricción especifica que la suma de los pesos de los objetos asignados al recipiente $j$ no supere su capacidad.

Vamos a formular este modelo empleando el API Python de Gurobi. Empezamos por definir los conjuntos de índices, que en este caso serán del tipo `tuplelist`. Para usar este tipo es necesario importar primero el módulo de Gurobi:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

n = 5
I = gp.tuplelist(range(1, n+1)) # indexamos los objetos como {1, ..., n}
J = gp.tuplelist(range(1, n+1)) # indexamos los recipientes como {1, ..., n}

print ('I:= {}'.format(I))
print ('J:= {}'.format(J))

Luego definimos los parámetros del modelo. Aquellos parámetros que dependen de índices se definen como diccionarios `tupledict`:

In [ ]:
# los pesos están dados por un diccionario cuyas claves son elementos de I
w = gp.tupledict({ 1 : 50, 
      2 : 45, 
      3 : 55, 
      4 : 40, 
      5 : 48})
b = 100 # capacidad de los recipientes

El siguiente paso es crear el objeto modelo:

In [ ]:
m = gp.Model('bin-packing')

Para crear las variables, podemos usar el método `addVars`. Como ya lo señalamos en el Cuaderno 2, este método permite crear una familia de variables indexadas por uno o más objetos del tipo `tuplelist`. En nuestro caso, las variables $x_{ij}$ están indexadas por todas las combinaciones de objetos $i \in I$ y recipientes $j \in J$. Por lo tanto, los dos primeros argumentos de la función serán `I` y `J`:

In [ ]:
# creamos variables x indexadas por I x J
x = m.addVars(I, J, vtype = GRB.BINARY, name="x")

Los demás argumentos de la función son los mismos que los argumentos de `addVar`. Recordar que el argumento `name` en este caso no fija el nombre de una variable, sino un nombre genérico que se utilizará como prefijo para el nombre de cada variable individual, y que será completado con los valores de los índices.

Recordar que la función `addVars` retorna un objeto del tipo `tupledict` con todas las variables:

In [ ]:
# antes de consultar el contenido de cualquier variable, es necesario llamar a update()
m.update()
# x es del tipo tupledict
print (type(x))
print (x)
# las claves de x forman un tuplelist
print (x.keys())
# para acceder a un elemento, hay que usar su tupla como índice; los paréntesis pueden obviarse
# "x" se usa como prefijo para el nombre de la variable
print(x[1,2].varName)


De manera similar, creamos las variables $y_j$ indexadas por J:

In [ ]:
y = m.addVars(J, vtype = GRB.BINARY, name="y")

Para construir la función objetivo, podemos usar el método `sum` de la clase `tupledict`:

In [ ]:
# la función objetivo es la suma de las variables
m.setObjective(y.sum('*'), GRB.MINIMIZE)

Definimos ahora las restricciones:

1. Cada objeto es empacado en un recipiente: para expresar esta restricción, podemos emplear el método `sum` de la clase `tupledict`, con un criterio de selección `(i, '*')`. Notar que hay una restricción para cada $i \in I$, por lo que empleamos el método `addConstrs` con una expresión generadora que depende de `i`:

In [ ]:
# Cada objeto es asignado a un recipiente
m.addConstrs((x.sum(i, '*') == 1 for i in I), "asig") 

2. Se asignan objetos únicamente a recipientes utilizados, y respetando su capacidad: En este caso utilizamos la función `quicksum`, conjuntamente con un generador que depende de `i`, para construir la expresión $\sum_i w_i x_{ij}$. Notar además que hay una restricción para cada $j \in J$, por lo que empleamos el método `addConstrs` y anidamos la expresión generadora anterior en una nueva expresión que depende de `j`:

In [ ]:
# Se asignan objetos únicamente a recipientes utilizados, y respetando su capacidad
m.addConstrs((gp.quicksum(w[i] * x[i, j] for i in I) <= b * y[j] 
              for j in J), "capac")

Finalmente, optimizamos el modelo:

In [ ]:
m.optimize()

Terminada la optimización, podemos mostrar las soluciones. Notar que se usa la sintaxis `x[i, j]` para acceder a la variable $x_{ij}$, pues `x` es un diccionario cuyas claves son pares ordenados del conjunto $I \times J$. Recordar que la propiedad `x` de un objeto variable nos permite acceder a su valor en la solución, indistintamente del nombre del objeto.

Notar que al comparar si los valores de las variables binarias son iguales a 1 usamos la condición `>=0.99 ` en lugar de la condición `== 1`, pese que se trata de variables enteras. El motivo es que Gurobi (al igual que otros solvers) trabaja internamente con valores de punto flotante (decimales) para todas las variables del modelo, debido a que debe resolver relajaciones lineales del programa entero. Gurobi asume que una variable tiene un valor entero cuando su valor está dentro de un rango de tolerancia del entero más próximo. Este rango está fijado en el parámetro `IntFeasTol` y su valor por defecto es $10^{-5}$. Más información puede consultarse en [este artículo](https://support.gurobi.com/hc/en-us/articles/360012237872-Why-does-Gurobi-sometimes-return-values-for-integer-variables-that-are-not-integers-).

In [ ]:
for j in J:
    # y[j].x es el valor de la variable y_j en la solución
    if y[j].x >= 0.99:
        print("Se usó el recipiente {}.".format(j))
        
for i in I:
    for j in J:
        # x[i,j].x es el valor de la variable x_{ij} en la solución
        if x[i, j].x >= 0.99:
            print("Objeto {} asignado al recipiente {}.".format(i, j))

## Código completo

Reproducimos a continuación el código completo del ejemplo:

In [ ]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de bin packing
# EPN (2024)

import gurobipy as gp
from gurobipy import GRB
import random as rd
try:
    n = 30
    I = gp.tuplelist(range(1, n+1)) # indexamos los objetos como {1, ..., n}
    J = gp.tuplelist(range(1, n+1)) # indexamos los recipientes como {1, ..., n}

    # los pesos están dados por un diccionario cuyas claves son elementos de I
    w = gp.tupledict({i : rd.randint(30,60) for i in I})
    b = 100 # capacidad de los recipientes

    m = gp.Model('bin-packing')

    # creamos variables x indexadas por I x J
    x = m.addVars(I, J, vtype = GRB.BINARY, name="x")

    # creamos variables y indexadas por J
    y = m.addVars(J, vtype = GRB.BINARY, name="y")

    # la función objetivo es la suma de las variables
    m.setObjective(y.sum('*'), GRB.MINIMIZE)

    # Restricciones
    # Cada objeto es asignado a un recipiente
    m.addConstrs((x.sum(i, '*') == 1 for i in I), "asig") 

    # Se asignan objetos únicamente a recipientes utilizados, y respetando su capacidad
    m.addConstrs((gp.quicksum(w[i] * x[i, j] for i in I) <= b * y[j] for j in J), "capac")
    
    # Resolver el modelo
    m.optimize()

    # Mostrar la solución
    for j in J:
        if y[j].x >= 0.99:
            print("Se usó el recipiente {}.".format(j))
        
    for i in I:
        for j in J:
            if x[i, j].x >= 0.99:
                print("Objeto {} asignado al recipiente {}.".format(i, j))

except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')